# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [7]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
import re
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.decomposition import TruncatedSVD
import pickle

nltk.download(['punkt', 'wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('msgs_tbl', engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):
    text  =  re.sub(r'[^a-zA-Z0-9]', ' ', text.lower())
    words = word_tokenize(text)
    words = [w for w in words if w not in stopwords.words('english')]
    tokens = [WordNetLemmatizer().lemmatize(w.strip()) for w in words]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
forest_clf = RandomForestClassifier()
pipeline = Pipeline([('tfidf', TfidfVectorizer()), ('clf', MultiOutputClassifier(forest_clf))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size = 0.20, random_state = 36)
pipeline.fit(xtrain, ytrain)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
ypreds = pipeline.predict(xtest)

In [13]:
for ind, col in enumerate(ytest):
    print(col)
    print(classification_report(ytest[col], ypreds[:,ind]))

related
             precision    recall  f1-score   support

          0       0.63      0.36      0.46      1205
          1       0.83      0.94      0.88      3999
          2       0.50      0.44      0.47        32

avg / total       0.78      0.80      0.78      5236

request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      4318
          1       0.82      0.36      0.50       918

avg / total       0.87      0.87      0.85      5236

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5220
          1       0.00      0.00      0.00        16

avg / total       0.99      1.00      1.00      5236

aid_related
             precision    recall  f1-score   support

          0       0.72      0.87      0.79      3044
          1       0.74      0.53      0.62      2192

avg / total       0.73      0.73      0.72      5236

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
pipeline.get_params()

{'memory': None,
 'steps': [('tfidf',
   TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
           stop_words=None, strip_accents=None, sublinear_tf=False,
           token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
           vocabulary=None)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None, verbose=0,
               warm_start=False),
         

In [15]:
parameters = {
    'clf__estimator__n_estimators': [10, 20],
    'clf__estimator__min_samples_split': [2, 4]
}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1)
cv.fit(xtrain, ytrain)
cv.best_estimator_

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
ypreds = cv.predict(xtest)

In [17]:
for ind, col in enumerate(ytest):
    print(col)
    print(classification_report(ytest[col], ypreds[:,ind]))

related
             precision    recall  f1-score   support

          0       0.70      0.33      0.45      1205
          1       0.82      0.95      0.88      3999
          2       0.44      0.53      0.48        32

avg / total       0.79      0.81      0.78      5236

request
             precision    recall  f1-score   support

          0       0.88      0.99      0.93      4318
          1       0.88      0.39      0.54       918

avg / total       0.88      0.88      0.87      5236

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5220
          1       0.00      0.00      0.00        16

avg / total       0.99      1.00      1.00      5236

aid_related
             precision    recall  f1-score   support

          0       0.74      0.87      0.80      3044
          1       0.76      0.56      0.65      2192

avg / total       0.75      0.74      0.74      5236

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [31]:
pipeline_2 = Pipeline([
                    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
                    ('best', TruncatedSVD(n_components=50)),
                    ('clf', MultiOutputClassifier(forest_clf))
                      ])

parameters= {'clf__estimator__n_estimators': [10, 20], 'clf__estimator__min_samples_split': [100, 120]}

cv = GridSearchCV(pipeline_2, param_grid=parameters, n_jobs=-1, cv = 2)

from workspace_utils import active_session

with active_session(): 
    import time
    start = time.time()

    cv.fit(xtrain, ytrain)
    ypreds = cv.best_estimator_.predict(xtest)

    end = time.time()
    print("time taken: {}".format(end - start))


    for ind, col in enumerate(ytest):
        print(col)
        print(classification_report(ytest[col], ypreds[:,ind]))

time taken: 1590.8260526657104
related
             precision    recall  f1-score   support

          0       0.68      0.29      0.41      1205
          1       0.81      0.96      0.88      3999
          2       0.43      0.09      0.15        32

avg / total       0.78      0.80      0.77      5236

request
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      4318
          1       0.77      0.48      0.59       918

avg / total       0.88      0.88      0.87      5236

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5220
          1       0.00      0.00      0.00        16

avg / total       0.99      1.00      1.00      5236

aid_related
             precision    recall  f1-score   support

          0       0.75      0.83      0.79      3044
          1       0.72      0.61      0.66      2192

avg / total       0.74      0.74      0.73      5236

medical_help
         

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [32]:
with open('classifier.pkl', 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
# import packages
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
import re
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.decomposition import TruncatedSVD
import pickle
from workspace_utils import active_session

nltk.download(['punkt', 'wordnet', 'stopwords'])

def load_data(db_filepath):
    """
    Loads data from database and returns X and Y.
    Args:
      db_filepath(str): string filepath of the sqlite database
    Returns:
      X(pandas DataFrame): messages
      Y(pandas DataFrame): labels
    """
    
    # read in file
    engine = create_engine('sqlite:///'+db_filepath)
    df = pd.read_sql_table('msgs_tbl', engine, if_exists='replace')
    engine.dispose()
    
    # define features and label arrays
    X = df['message']
    Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

    return X, Y

def tokenize(text):
    """
    Tokenizes normalized text
    Args:  
        text(string): input text
    Return:
        tokens(list of strings): list of cleaned tokens
    
    """
    #normalizing text: removing punctuations and converting to lowercase
    text  =  re.sub(r'[^a-zA-Z0-9]', ' ', text.lower())
    
    #tokenizing text
    words = word_tokenize(text)
    
    #removing stop words
    words = [w for w in words if w not in stopwords.words('english')]
    
    #lemmatizing
    tokens = [WordNetLemmatizer().lemmatize(w.strip()) for w in words]
    
    return tokens

def build_model():
    """
    Returns a GridSearchCV object to be used as the model for training
    Args:
       None
    Returns:
       cv (GridSearchCV): Grid search model object
    """
    # text processing and model pipeline
    forest_clf = RandomForestClassifier()
    pipeline_2 = Pipeline([
                    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
                    ('best', TruncatedSVD(n_components=50)),
                    ('clf', MultiOutputClassifier(forest_clf))
                      ])

    # define parameters for GridSearchCV
    parameters= {'clf__estimator__n_estimators': [10, 20], 'clf__estimator__min_samples_split': [100, 120]}

    # create gridsearch object and return as final model pipeline
    cv = GridSearchCV(pipeline_2, param_grid=parameters, n_jobs=-1, cv = 2)

    return cv


def train(X, Y, cv):
    """
     Returns a model trained on training data and prints its classification report
     Args:
        X(pandas DataFrame): messages
        y(pandas DataFrame): labels
     Returns:
        cv : Grid search model object
     """
    # train test split
    xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size = 0.20, random_state = 36)
    
    with active_session():
        # fit model
        model = cv.fit(xtrain, ytrain)

        # output model test results
        ypreds = model.best_estimator_.predict(xtest)
        for ind, col in enumerate(ytest):
            print(col)
            print(classification_report(ytest[col], ypreds[:,ind]))
    
    return model


def save_model(model):
    """
    Saves model as pickle file.
    Args:
      cv:  model
    Return:
      N/A
    """
    with open('classifer.pkl', 'wb') as f:
        pickle.dump(model, f)  

def run_pipeline(data_file):
    print('Getting features and labels..')
    X, Y = load_data('DisasterResponse.db')  # run ETL pipeline
    print('Building model..')
    cv = build_model()  # build model pipeline
    print('Training Model..Printing classification report in a few moments..')
    model = train(X, Y, cv)  # train model pipeline
    print('Saving model as pickle..')
    save_model(model)  # save model
    print('Pipeline run successfull!')


if __name__ == '__main__':
    data_file = sys.argv[1]  # get filename of dataset
    run_pipeline(data_file)  # run data pipeline